In [ ]:
# default_exp core.database_rq

In [ ]:
#hide
%reload_ext autoreload
%autoreload 2

In [ ]:
#hide
from nbdev.showdoc import *
import json
import responses
from pathlib import Path
from discogspy.core import *

In [ ]:
#hide
with open(Path("../config/example_user_info.json"), "r") as user_config_file:
    config = json.load(user_config_file)
    user = UserWithUserTokenBasedAuthentication(user_token=config["user_token"],
                                                user_agent=config["user_agent"])

# Core Database Requests

> function wrapper around the discogs api for database interactions

In [ ]:
#export
import requests
from typing import Union
from discogspy.core import *

In [ ]:
#export
def get_release(user:UserWithUserTokenBasedAuthentication, 
                release_id:int, 
                curr_abbr=Union[str, None])->requests.models.Response:
    """
    Get information to a particular release from discogs database.
    A release represents a particular physical or digital object released by 
    one or more Artists.
    
    No user Authentication needed.
    """
    url = f"{RELEASES_URL}/{release_id}"
    headers = user.headers
    params = {"token": user.user_token} 
    if curr_abbr and curr_abbr in VALID_CURR_ABBR: 
        params["curr_addr"] = curr_abbr
    return requests.get(url, headers=headers, params=params)

In [ ]:
# Example for usage of get_releases
get_release(user=user, release_id=6067651, curr_abbr="EUR")

<Response [200]>

In [ ]:
#hide 
@responses.activate
def test_get_release():
    test_user = UserWithUserTokenBasedAuthentication(user_token="test_user_token",
                                                     user_agent="test_user_agent")

    target_url = f"{RELEASES_URL}/123456?token=test_user_token&curr_addr=EUR"
    test_content = {"TestRelease": "TestContent"}
    
    responses.add(responses.GET, target_url,
                  json=test_content, status=200)

    resp = get_release(user=test_user, release_id=123456, curr_abbr="EUR")

    assert resp.json() == test_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url

test_get_release()

In [ ]:
#export
def get_release_rating_by_user(user:UserWithUserTokenBasedAuthentication, 
                               release_id:int, 
                               username=str)->requests.models.Response:
    """
    Get the rating of a release for a given user.
    
    No user Authentication needed. 
    """
    url = f"{RELEASES_URL}/{release_id}/rating/{username}"
    headers = user.headers
    params = {"token": user.user_token} 
    return requests.get(url, headers=headers, params=params)

In [ ]:
# Example for usage of get_release_rating_by_user
get_release_rating_by_user(user=user, release_id=9453891, username="darkwolf")


<Response [200]>

In [ ]:
#hide 
@responses.activate
def test_release_rating_by_user():
    test_user = UserWithUserTokenBasedAuthentication(user_token="test_user_token",
                                                     user_agent="test_user_agent")
    target_url = f"{RELEASES_URL}/123456/rating/test_user_name?token=test_user_token"
    test_content = {"username": "test_user_name", "rating": 5, "release_id": 123456}
    
    responses.add(responses.GET, target_url,
                  json=test_content, status=200)

    resp = get_release_rating_by_user(user=test_user, release_id=123456, username="test_user_name")

    assert resp.json() == test_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url

test_release_rating_by_user()

In [ ]:
#hide
from nbdev.export import *
notebook2script()